In [1]:
df = read.csv('../datasets/mediause.csv')
model = lm(formula = 'newspaper ~ age + gender', data = df)
model


Call:
lm(formula = "newspaper ~ age + gender", data = df)

Coefficients:
(Intercept)          age       gender  
   -0.08956      0.06762      0.17666  


In [2]:
gender = c(1,0)
age = c(20,40)
newdata = data.frame(age, gender)
predict(model, newdata)

1        2 
1.439508 2.615248

In [66]:
library(tidyverse)
library(caret)
library(rsample)
library(e1071)
set.seed(42)
df = na.omit(df %>% mutate(usesinternet=recode(internet, .default=TRUE, `0`=FALSE)))
print("How many people used online news at all?")
print(table(df$usesinternet))
print("Let's see how well we can predict this...")

split = initial_split(df, prop = .8)
train = training(split)
test  = testing(split)

myclassifier=naiveBayes(usesinternet~age+gender+education, data=train)

testPred = predict(myclassifier, newdata=test, type="class")
print("Contingency Table for Test Data")
conttab = table(test$usesinternet, testPred)
print(conttab)

print('Precision for predicting True internet users and non-internet-users, respecitively:')
precision = diag(conttab) / rowSums(conttab)
print(precision)


print('Recall for predicting True internet users and non-internet-users, respecitively:')
recall = (diag(conttab) / colSums(conttab))
print(recall)

[1] "How many people used online news at all?"

FALSE  TRUE 
  803  1262 
[1] "Let's see how well we can predict this..."
[1] "Contingency Table for Test Data"
       testPred
        FALSE TRUE
  FALSE    40   97
  TRUE     57  218
[1] "Precision for predicting True internet users and non-internet-users, respecitively:"
    FALSE      TRUE 
0.2919708 0.7927273 
[1] "Recall for predicting True internet users and non-internet-users, respecitively:"
    FALSE      TRUE 
0.4123711 0.6920635 


In [67]:
print("To be added")

[1] "To be added"


In [68]:
print("To be added")

[1] "To be added"


In [69]:
print("To be added")

[1] "To be added"
